<a href="https://colab.research.google.com/github/AzlinRusnan/Optimizing-Customer-Satisfaction-CSAT-Through-Sentiment-Analysis-and-Predictive-ML-Techniques/blob/main/Optimizing_CSAT_Through_Sentiment_Analysis_%26_Predictive_ML_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

file_path = '/content/gdrive/MyDrive/CSAT RAW DATASET.xlsx'
xls = pd.ExcelFile(file_path)

# Check sheet names to understand the structure
xls.sheet_names

['Page 1']

In [6]:
# Load the data from the first sheet
df = pd.read_excel(xls, sheet_name='Page 1')

# Display the first few rows to understand the structure and locate the 'City' column
df.head()

,Number,Location,City,Country,Region,Updated,Average Response (calculated),USS Comment,String value
0,INC19296127,USPO,Pasco,United States of America,NORTH AMERICA,2024-10-31 23:18:02,1,\n\n\n\n,Very Satisfied
1,INC19297125,USGR,Greensboro,United States of America,NORTH AMERICA,2024-10-31 22:35:51,1,Thanks for punctual and quick service resolvin...,Very Satisfied
2,INC19283148,USGR,Greensboro,United States of America,NORTH AMERICA,2024-10-31 22:06:26,1,\n\n\n,Very Satisfied
3,INC19296794,BRSP,Sao Paulo,Brazil,LATAM,2024-10-31 21:39:18,1,\n\n\n,Very Satisfied
4,INC19295496,INPU,Pune,India,APAC,2024-10-31 21:26:21,1,\n\nPrompt response\n,Very Satisfied


In [12]:
# Check for missing values in the 'City' column
missing_values_city = df['City'].isnull().sum()
print("There are " + str(missing_values_city)  + " missing values in the City column")

There are 40380 missing values in the City column


In [14]:
# To fill the missing 'City' values, I will first explore how to handle it.
# One approach is to use the most frequent city in the respective 'Location' or 'Country'.

# Check for the distribution of cities in the dataset
city_distribution = df['City'].value_counts()

# Checking the first few rows of 'Location' and 'City' to explore patterns
df[['Location','City']].head(10)

,Location,City
0,USPO,Pasco
1,USGR,Greensboro
2,USGR,Greensboro
3,BRSP,Sao Paulo
4,INPU,Pune
5,INPU,Pune
6,INPU,Pune
7,BRIF,São Paulo
8,MXMO,Los Mochis
9,GBGU,Guildford


In [19]:
# I will attempt to fill missing 'City' values based on the most frequent city in each 'Location'.
city_mapping = df.groupby('Location')['City'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown').to_dict()

# Apply this mapping to fill missing 'City' values
df['City'] = df['City'].fillna(df['Location'].map(city_mapping))

# Check the number of missing values after filling
missing_values_after = df['City'].isnull().sum()
print("There are now " + str(missing_values_after) + " missing values in City!")

There are now 0 missing values in City!


In [20]:
# Since we have fill in the gap in City. Lets proceed with missing values in String Value

# Check for missing values in the 'City' column
missing_values_sv = df['String value'].isnull().sum()
print("There are " + str(missing_values_sv)  + " missing values in the String Value column")

There are 41040 missing values in the String Value column


In [21]:
# Let's check if there is any pattern to help us fill missing values in the 'String value' column
df[['String value']].drop_duplicates().head(20)

,String value
0,Very Satisfied
20,Satisfied
50,Very Dissatisfied
55,Neutral
70,Dissatisfied
49648,NaN


In [22]:
# Create a mapping for the "Average Response" to the corresponding "String value"
response_to_string_map = {
    1: "Very Satisfied",
    2: "Satisfied",
    3: "Neutral",
    4: "Dissatisfied",
    5: "Very Dissatisfied"
}

# Get the most frequent value in the 'Average Response (calculated)' column
most_frequent_response = df['Average Response (calculated)'].mode().iloc[0]

# Map the most frequent 'Average Response' to its corresponding 'String value'
most_frequent_string = response_to_string_map[most_frequent_response]

# Fill missing 'String value' with the most frequent 'String value' based on 'Average Response'
df['String value'] = df['String value'].fillna(most_frequent_string)

# Check if there are still any missing values in 'String value'
missing_values_string_after = df['String value'].isnull().sum()
missing_values_string_after


0